In [ ]:

1pip install

In [ ]:
from __future__ import print_function
from tensorflow.keras.models import load_model
import numpy as np
import time
import os
import math
import cyipopt as ipopt
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import joblib
from joblib import Parallel, delayed
# Import tqdm
from tqdm import tqdm

# --- Load Model, Scaler Object, and Recreate Encoder ---
# (Same as before)
drive_path = './'
model_path = os.path.join(drive_path, 'gruGA_datacombID.keras')
scaler_path = os.path.join(drive_path, 'scaler.pkl')

try:
    model = load_model(model_path)
    print(f"Model loaded successfully from {model_path}")
except Exception as e:
    print(f"Error loading Keras model: {e}")
    exit()

try:
    scaler = joblib.load(scaler_path)
    print(f"Scaler loaded successfully from {scaler_path}")
    full_mean = scaler.mean_
    full_scale = scaler.scale_
    Xphys_mean = full_mean[:21]
    Xphys_std = full_scale[:21]
    y_mean = full_mean[21:]
    y_std = full_scale[21:]
except FileNotFoundError:
    print(f"Error: Scaler file not found at {scaler_path}. Please ensure it exists.")
    exit()
except Exception as e:
    print(f"Error loading scaler: {e}")
    exit()

NUM_TRAJECTORIES = 10
encoder = OneHotEncoder(sparse_output=False)
traj_ids = np.arange(NUM_TRAJECTORIES).reshape(-1, 1)
encoder.fit(traj_ids)

# --- MPC Parameters ---
# (Same as before)
delta = 0.01
NUM_MPC_ITERATION = 100 # Max iterations to attempt
NUM_PHYSICAL_FEATURES = 21
NUM_GRU_INPUT_FEATURES = NUM_PHYSICAL_FEATURES + NUM_TRAJECTORIES # 31
NUM_STATE_VARIABLES = 14
NUM_CONTROL_INPUTS = 7
NUM_GRU_OUTPUTS = 14
SEQ_LENGTH = 10
HORIZON = 5
NUM_MPC_OPTIMIZATION_VARS = NUM_CONTROL_INPUTS * HORIZON
NUM_MPC_CONSTRAINTS = 0

# --- Reference Trajectory ---
# (Same as before)
ref_traj_file = os.path.join(drive_path, 'kukatraj10.csv')
try:
    ref_df = pd.read_csv(ref_traj_file)
    reference_trajectory_full = ref_df.values
    print(f"Reference trajectory loaded from {ref_traj_file}, shape: {reference_trajectory_full.shape}")
except FileNotFoundError:
    print(f"Error: Reference trajectory file not found at {ref_traj_file}")
    exit()
except Exception as e:
    print(f"Error reading reference trajectory: {e}")
    exit()

ref_traj_id = 9
ref_traj_one_hot = encoder.transform([[ref_traj_id]])
reference_states_full = reference_trajectory_full[:, 21:35]

# --- Initial State ---
# (Same as before)
if len(reference_trajectory_full) < SEQ_LENGTH:
    print(f"Error: Reference trajectory length ({len(reference_trajectory_full)}) is less than sequence length ({SEQ_LENGTH}).")
    exit()
initial_state = reference_trajectory_full[SEQ_LENGTH - 1, 0:14]
initial_sequence_physical = reference_trajectory_full[0:SEQ_LENGTH, 0:21]
initial_sequence_onehot = np.tile(ref_traj_one_hot, (SEQ_LENGTH, 1))
current_gru_input_sequence = np.hstack((initial_sequence_physical, initial_sequence_onehot))

# --- Global variables ---
# (Same as before)
global_current_state = initial_state.copy()
global_current_gru_input_sequence = current_gru_input_sequence.copy()
mpc_step_index = 0

# --- Manual Scaling/Inverse Scaling Functions ---
# (Same as before)
def scale_physical_inputs(data_phys_unscaled, mean_vec, std_vec):
    return (data_phys_unscaled - mean_vec) / (std_vec + 1e-8)

def inverse_scale_outputs(data_scaled, mean_vec, std_vec):
    return data_scaled * (std_vec + 1e-8) + mean_vec

# --- GRU Prediction Function (Helper for callbacks) ---
# (Same as before)
def predict_horizon(current_input_sequence_unscaled, control_sequence_horizon_unscaled, ref_one_hot):
    predicted_states_horizon_unscaled = np.zeros((HORIZON, NUM_GRU_OUTPUTS))
    temp_input_sequence_unscaled = current_input_sequence_unscaled.copy()
    reshaped_controls_unscaled = control_sequence_horizon_unscaled.reshape(HORIZON, NUM_CONTROL_INPUTS)
    for i in range(HORIZON):
        physical_part_unscaled = temp_input_sequence_unscaled[:, :NUM_PHYSICAL_FEATURES]
        one_hot_part = temp_input_sequence_unscaled[:, NUM_PHYSICAL_FEATURES:]
        scaled_physical_part = scale_physical_inputs(physical_part_unscaled, Xphys_mean, Xphys_std)
        scaled_input_sequence_step = np.hstack((scaled_physical_part, one_hot_part))
        gru_input = scaled_input_sequence_step.reshape(1, SEQ_LENGTH, NUM_GRU_INPUT_FEATURES)
        scaled_predicted_output = model.predict(gru_input, verbose=0)
        predicted_output_unscaled = inverse_scale_outputs(scaled_predicted_output, y_mean, y_std)
        predicted_states_horizon_unscaled[i, :] = predicted_output_unscaled[0]
        next_pos_vel_unscaled = predicted_output_unscaled[0]
        current_torques_unscaled = reshaped_controls_unscaled[i, :]
        next_physical_features_unscaled = np.concatenate((next_pos_vel_unscaled, current_torques_unscaled))
        next_full_input_step_unscaled = np.concatenate((next_physical_features_unscaled, ref_one_hot[0]))
        temp_input_sequence_unscaled = np.vstack((temp_input_sequence_unscaled[1:], next_full_input_step_unscaled))
    return predicted_states_horizon_unscaled

# --- Define the Problem Class for cyipopt ---
# (Same as before, including parallel gradient)
class MPCProblem:
    def __init__(self, n_jobs=-1):
        self.n_jobs = n_jobs
        self.finite_diff_step = 1e-4

    def objective(self, x_unscaled):
        global global_current_gru_input_sequence, reference_states_full, mpc_step_index, ref_traj_one_hot
        predicted_states_unscaled = predict_horizon(global_current_gru_input_sequence, x_unscaled, ref_traj_one_hot)
        start_ref_idx = mpc_step_index + 1
        end_ref_idx = start_ref_idx + HORIZON
        actual_horizon_len = HORIZON
        if end_ref_idx > len(reference_states_full):
            end_ref_idx = len(reference_states_full)
            actual_horizon_len = end_ref_idx - start_ref_idx
            if actual_horizon_len < HORIZON and actual_horizon_len >= 0:
                 predicted_states_unscaled = predicted_states_unscaled[:actual_horizon_len, :]
            elif actual_horizon_len < 0:
                 return 0.0
        if actual_horizon_len <= 0: return 0.0
        reference_states_horizon_unscaled = reference_states_full[start_ref_idx:end_ref_idx, :]
        if predicted_states_unscaled.shape[0] != reference_states_horizon_unscaled.shape[0]:
            min_len = min(predicted_states_unscaled.shape[0], reference_states_horizon_unscaled.shape[0])
            if min_len <= 0: return 0.0
            cost = np.sum((predicted_states_unscaled[:min_len, :] - reference_states_horizon_unscaled[:min_len, :])**2)
        else:
            cost = np.sum((predicted_states_unscaled - reference_states_horizon_unscaled)**2)
        return cost

    def gradient(self, x_unscaled):
        n = len(x_unscaled)
        step = self.finite_diff_step
        def compute_obj_diff(i, x_base):
            x_plus = x_base.copy()
            x_minus = x_base.copy()
            x_plus[i] += step
            x_minus[i] -= step
            f_plus = self.objective(x_plus)
            f_minus = self.objective(x_minus)
            return f_plus - f_minus
        diffs = Parallel(n_jobs=self.n_jobs)(delayed(compute_obj_diff)(i, x_unscaled) for i in range(n))
        grad = np.array(diffs) / (2 * step)
        return grad

    def constraints(self, x_unscaled):
        assert NUM_MPC_CONSTRAINTS == 0, "Constraint function not implemented"
        return np.array([])

    def jacobianstructure(self):
        assert NUM_MPC_CONSTRAINTS == 0, "Jacobian structure not needed"
        return (np.array([], dtype=int), np.array([], dtype=int))

    def jacobian(self, x_unscaled):
        assert NUM_MPC_CONSTRAINTS == 0, "Jacobian calculation not needed"
        return np.array([])

# --- IPOPT Setup ---
# (Same as before)
nvar = NUM_MPC_OPTIMIZATION_VARS
ncon = NUM_MPC_CONSTRAINTS
torque_lower_limit = -10.0
torque_upper_limit = 10.0
lb = np.full(nvar, torque_lower_limit)
ub = np.full(nvar, torque_upper_limit)
cl = np.array([])
cu = np.array([])

# --- MPC Loop ---
actual_states_history = [initial_state]
applied_torques_history = []
mpc_computation_time = []
x0 = np.zeros(nvar)

print("Starting MPC...")

min_required_len = SEQ_LENGTH + HORIZON
if len(reference_trajectory_full) < min_required_len:
     print(f"Error: Reference trajectory too short ({len(reference_trajectory_full)}) for sequence length ({SEQ_LENGTH}) and horizon ({HORIZON}). Needs {min_required_len} steps.")
     total_steps = 0
else:
    max_possible_steps = len(reference_trajectory_full) - SEQ_LENGTH - HORIZON + 1
    total_steps = min(NUM_MPC_ITERATION, max_possible_steps)

print(f"Calculated total MPC steps: {total_steps}")
num_cores_to_use = -1
print(f"Using n_jobs={num_cores_to_use} for parallel gradient calculation.")

# Initialize tqdm progress bar before the loop [6]
pbar = tqdm(total=total_steps, desc="Starting MPC...")

try: # Use try...finally to ensure pbar is closed
    for k in range(total_steps):
        # Update global step index for callbacks
        mpc_step_index = k

        # Update tqdm description to show current iteration [8]
        pbar.set_description(f"MPC Iter {k}/{total_steps} (Solving)")

        start_time = time.time()
        global_current_gru_input_sequence = current_gru_input_sequence

        # --- Create cyipopt Problem ---
        problem_obj = MPCProblem(n_jobs=num_cores_to_use)
        nlp = ipopt.Problem(n=nvar, m=ncon, problem_obj=problem_obj, lb=lb, ub=ub, cl=cl, cu=cu)

        # --- Set IPOPT Options ---
        nlp.add_option('max_iter', 200)
        nlp.add_option('tol', 1e-4)
        nlp.add_option('print_level', 0) # Keep low to avoid interfering with tqdm
        nlp.add_option('hessian_approximation', 'limited-memory')

        # --- Solve the Optimization Problem ---
        try:
            x_opt_unscaled, info = nlp.solve(x0)
            status = info['status']
            obj = info['obj_val']
        except Exception as e:
            print(f"\n!!! IPOPT solve failed in iteration {k} with exception: {e}")
            print("Stopping MPC loop.")
            break # Exit the main loop

        end_time = time.time()
        iter_time = end_time - start_time
        mpc_computation_time.append(iter_time)

        # Update tqdm postfix with last iteration time and solver status [6]
        status_msg = info['status_msg'].decode()
        pbar.set_postfix_str(f"Last Time: {iter_time:.3f}s, Status: {status_msg}")

        # (Optional) Print warning if solver status is not optimal/acceptable
        if status < 0:
             print(f"\nWarning: IPOPT finished iteration {k} with status {status} ({status_msg}).")

        # --- Extract controls and Simulate ---
        pbar.set_description(f"MPC Iter {k}/{total_steps} (Simulating)") # Update description

        optimal_torques_unscaled = x_opt_unscaled[0:NUM_CONTROL_INPUTS]
        applied_torques_history.append(optimal_torques_unscaled)

        # (Simulation code remains the same)
        current_physical_input_unscaled = np.concatenate((global_current_state, optimal_torques_unscaled))
        current_full_input_step_unscaled = np.concatenate((current_physical_input_unscaled, ref_traj_one_hot[0]))
        sim_input_sequence_unscaled = np.vstack((current_gru_input_sequence[1:], current_full_input_step_unscaled))
        sim_physical_part_unscaled = sim_input_sequence_unscaled[:, :NUM_PHYSICAL_FEATURES]
        sim_one_hot_part = sim_input_sequence_unscaled[:, NUM_PHYSICAL_FEATURES:]
        sim_scaled_physical_part = scale_physical_inputs(sim_physical_part_unscaled, Xphys_mean, Xphys_std)
        sim_scaled_input_sequence = np.hstack((sim_scaled_physical_part, sim_one_hot_part))
        gru_sim_input = sim_scaled_input_sequence.reshape(1, SEQ_LENGTH, NUM_GRU_INPUT_FEATURES)
        scaled_next_state = model.predict(gru_sim_input, verbose=0)
        next_state_unscaled = inverse_scale_outputs(scaled_next_state, y_mean, y_std)[0]
        global_current_state = next_state_unscaled
        current_gru_input_sequence = sim_input_sequence_unscaled
        actual_states_history.append(global_current_state)

        # --- Update initial guess (Warm Start) ---
        x0 = np.roll(x_opt_unscaled, -NUM_CONTROL_INPUTS)
        x0[-NUM_CONTROL_INPUTS:] = x_opt_unscaled[-NUM_CONTROL_INPUTS:]

        # --- Update the progress bar for the completed iteration ---
        pbar.update(1)

finally:
    pbar.close() # Ensure the progress bar is closed cleanly [6]

print("\nMPC Finished.")

# --- Analysis ---
# (Plotting code remains the same)
if mpc_computation_time:
    print(f"Average MPC computation time per iteration: {np.mean(mpc_computation_time):.4f} seconds")

if len(actual_states_history) > 1:
    actual_states_history = np.array(actual_states_history)
    applied_torques_history = np.array(applied_torques_history)

    plt.figure(figsize=(15, 10))
    num_joints_to_plot = 7
    plot_time_steps = len(actual_states_history)
    time_axis = np.arange(plot_time_steps)
    ref_start_idx = SEQ_LENGTH - 1
    ref_indices = np.arange(ref_start_idx, ref_start_idx + plot_time_steps)

    for i in range(num_joints_to_plot):
        plt.subplot(num_joints_to_plot, 1, i+1)
        plt.plot(time_axis, actual_states_history[:, i], label=f'Actual Joint {i+1} Pos', linewidth=2)
        valid_ref_indices = ref_indices[ref_indices < len(reference_trajectory_full)]
        ref_time_axis = time_axis[:len(valid_ref_indices)]
        plt.plot(ref_time_axis, reference_trajectory_full[valid_ref_indices, i], '--', label=f'Reference Joint {i+1} Pos', linewidth=2)
        plt.ylabel(f'Joint {i+1}')
        plt.grid(True)
        if i == 0:
            plt.title('MPC Tracking Performance (Unscaled Joint Positions)')
            plt.legend(loc='upper right')
        if i == num_joints_to_plot - 1:
            plt.xlabel('Time Step (k)')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 6))
    torque_time_axis = np.arange(len(applied_torques_history))
    for i in range(NUM_CONTROL_INPUTS):
         plt.plot(torque_time_axis, applied_torques_history[:, i], label=f'Torque Joint {i+1}')
    plt.title('Applied Unscaled Torques by MPC')
    plt.xlabel('Time Step (k)')
    plt.ylabel('Torque')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("MPC did not complete enough steps to generate plots.")


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Model loaded successfully from ./gruGA_datacombID.keras
Scaler loaded successfully from ./scaler.pkl
Reference trajectory loaded from ./kukatraj10.csv, shape: (1996, 35)
Starting MPC...
Calculated total MPC steps: 100
Using n_jobs=-1 for parallel gradient calculation.


MPC Iter 0/100 (Solving):   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
MPC Iter 1/100 (Solving):   1%|          | 1/100 [2:17:00<226:03:20, 8220.21s/it, Last Time: 8220.102s, Status: Maximum number of iterations exceeded (can be specified by an option).]   

MPC Iter 1/100 (Simulating):   2%|▏         | 2/100 [4:30:59<220:52:19, 8113.67s/it, Last Time: 8038.889s, Status: Maximum number of iterations exceeded (can be specified by an option).]

MPC Iter 2/100 (Solving):   2%|▏         | 2/100 [4:30:59<220:52:19, 8113.67s/it, Last Time: 8038.889s, Status: Maximum number of iterations exceeded (can be specified by an option).]   